In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
import re

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

# Initialiser lemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Régis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Régis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Régis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Régis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
def lemmatize_word(word):
    return lemmatizer.lemmatize(word, pos=wordnet.VERB)  # Lemmatisation des verbes

stop_words = set(stopwords.words("english"))
def remove_stopwords(text):
    words = word_tokenize(text.lower())  # Tokenisation du texte
    filtered_words = [word for word in words if word not in stop_words]  # Filtrer les stopwords
    return " ".join(filtered_words)

In [3]:
TabAttacks = pd.DataFrame(columns=['id', 'nom', 'description'])

In [4]:
url = "https://capec.mitre.org/data/slices/1000.html"
response = requests.get(url)
response.encoding = 'utf-8'

soup = BeautifulSoup(response.text, "html.parser")

In [5]:
listeH2= soup.find_all("h2")

In [6]:
listeID = []
for i in range(len(listeH2)):
    res = listeH2[i].text.split(":")[0].split('-')
    if len(res) > 1:
        listeID.append(res[1])

In [7]:
listeID = []
listeNom = []
for i in range(len(listeH2)):
    res = listeH2[i].text.split(":")[0].split('-')
    res2 = listeH2[i].text.split(":")[1]
    if len(res) > 1:
        listeID.append(res[1])
        listeNom.append(res2[1:])

In [8]:
listeDescription = soup.find_all("div", {"id": "Description"})

In [9]:
listeDescriptionText = []
OriginDescription = []
for i in range(len(listeDescription)):
    text = listeDescription[i].find("div", {"class": "indent"}).text
    OriginDescription.append(text)
    cleaned_text = remove_stopwords(text)
    listeDescriptionText.append(cleaned_text)

In [10]:
OriginDescription[24]

'An adversary modifies file contents by adding data to files for several reasons. Many different attacks could “follow” this pattern resulting in numerous outcomes. Adding data to a file could also result in a Denial of Service condition for devices with limited storage capacity. '

In [11]:
capec_dict_by_id = {
    1: "Improperly specified ACL",
    10: "Untrustable environment variables",
    100: "Improper bounds checking on buffer operations",
    101: "Untested code input",
    102: "Unencrypted communication channel",
    105: "Manipulable HTTP Request messages",
    107: "Unprotected HTTP header",
    108: "Unvalidated command input",
    128: "Numeric data handling",
    131: "System resources",
    163: "email account of some trusted entity",
    221: "Data serialization entity replacement",
    224: "Fingerprinting",
    243: "HTML attributes",
    268: "Log injection",
    272: "Communication protocol vulnerabilities",
    300: "Network port states",
    307: "RPC service enumeration",
    331: "UDP error message analysis",
    332: "ICMP error message behaviors",
    36: "Exposed system interfaces",
    37: "Embedded sensitive data",
    406: "Discarded sensitive information",
    429: "Eye movement analysis",
    448: "DLL injection",
    458: "Flash memory compromise",
    462: "Cross-domain request timing",
    464: "Persistent cookie storage",
    473: "Message forgery",
    52: "Null byte injection",
    531: "Hardware substitution attack",
    533: "Software update",
    535: "Hardware components",
    541: "Application Fingerprinting",
    542: "Targeted Malware",
    543: "Web site",
    549: "Target system",
    550: "Service operating software",
    560: "Weak exposed credentials",
    569: "Input capture mechanisms",
    57: "SSL termination vulnerability",
    571: "Log delivery failure",
    581: "Security tool enumeration",
    584: "BGP route advertisement suppression",
    591: "Reflected Cross-Site Scripting (XSS) vulnerability",
    599: "Satellite dish signal disruption",
    620: "Insecure encryption downgrade",
    642: "Insufficient file system permissions",
    694: "System information leakage",
    698: "Malicious extension installation",
    75: "Unprotected configuration files",
    94: "Insecure communication channel"
}

In [12]:

rule1 = [lemmatize_word(w) for w in [
    "spoofing", "phishing", "overlay", "squatting", "monitoring", "bypass",
    "abuse", "disabling", "seizure", "jamming", "blocking", "misuse", 
    "exploitation", "spoofing of", "exploiting incorrectly", "collect data from",
    "pretexting via", "bypassing of", "manipulate", "leveraging", "manipulating", 
    "disabling", "accessing", "intercepting", "modifying", "counterfeit", "fake the", 
    "exploit", "using", "leverage", "bypassing", "poison", "infected", "contaminate", 
    "detect", "capture", "sniffing"
]]

rule2 = [lemmatize_word(w) for w in [
    "flood", "tampering", "overflow", "poisoning", "alteration", "analysis",
    "impersonation", "manipulation", "expansion", "linking", "blowup",
    "fragmentation", "altered", "modification of"
]]

rule3 = [lemmatize_word(w) for w in [
    "hijacking", "splitting", "smuggling", "injection", "pollution", "inclusion",
    "insertion", "scanning", "discovery", "footprinting", "probe"
]]

def extract_va(text):
    words = word_tokenize(text.lower())  
    words = [lemmatize_word(w) for w in words]  

    for j, word in enumerate(words):
        if word in rule3 and j > 0:
            return f"Untested {words[j - 1]} Input"
        if word in rule2 and j >= 3:
            return " ".join(words[j - 3:j])
        if word in rule1 and j + 3 < len(words):
            return " ".join(words[j + 1:j + 4])
    return ""

va_results = [extract_va(desc) for desc in listeDescriptionText]

df_va = pd.DataFrame({
    "ID": listeID,
    "VA": va_results
})


In [13]:
print(capec_dict_by_id[10])

Untrustable environment variables


In [14]:
test = 2

In [15]:
df_va.loc[df_va['ID'] == str(test), 'VA']

229    security functionality system
Name: VA, dtype: object

In [16]:
cpt = 0
for i in range(len(df_va)):
    id = df_va.loc[i,'ID']
    id = int(id)
    if id in capec_dict_by_id.keys():
        df_va.loc[df_va['ID'] == str(id), 'VA'] = capec_dict_by_id[id]

In [17]:
listeDescriptionText[4]

"adversary able disguise one action another therefore trick user initiating one type action intend initiate different action . example , user might led believe clicking button submit query , fact downloads software . adversaries may perform attack social means , simply convincing victim perform action relying user 's natural inclination , technical means , clickjacking attack user sees one interface actually interacting second , invisible , interface ."

In [18]:
df_va[df_va['ID'] == "173"]

,ID,VA
4,173,


In [19]:
vides = df_va[df_va['VA'] == ""]

In [20]:
vides.index

Index([  4,   6,  29,  39,  60,  61,  62,  66,  84,  91,  96, 114, 126, 145,
       162, 177, 191, 214, 239, 272, 293, 306, 311, 322, 328, 329, 337, 344,
       360, 366, 392, 425, 442, 443, 452, 464, 465, 485, 513, 518],
      dtype='int64')

In [21]:
for i in range(len(vides)):
        listeDescriptionText[vides.index[i]]

In [22]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
# La question à poser au modèle BERT
question = "What is the vulnerable asset targeted by the attacker?"

res = []

for i in range(len(vides)):
    description = listeDescriptionText[vides.index[i]]

    if description.strip(): 
    
        vectorizer = TfidfVectorizer(stop_words='english', max_features=20)
        tfidf_matrix = vectorizer.fit_transform([description])
        feature_names = vectorizer.get_feature_names_out()

        nmf_model = NMF(n_components=1, random_state=42)
        nmf_topics = nmf_model.fit_transform(tfidf_matrix)
        
        topic = " ".join([feature_names[i] for i in nmf_model.components_[0].argsort()[:-6 - 1:-1]])

        inputs = tokenizer.encode_plus(question, topic, return_tensors="pt")
        outputs = model(**inputs)
        start_index = outputs.start_logits.argmax()
        end_index = outputs.end_logits.argmax()
        answer = tokenizer.convert_tokens_to_string(
            tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_index:end_index+1])
        )
        
        df_va.iloc[vides.index[i],1] = answer


c:\Users\Régis\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_nmf.py:133: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(res * 2)
c:\Users\Régis\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
c:\Users\Régis\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_nmf.py:133: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(res * 2)
c:\Users\Régis\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_nmf.py:133: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(res * 2)
c:\Users\Régis\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
c:\Us

In [24]:
def nettoyer_texte(texte):
    return re.sub(r"[.,:()]", '', texte)

In [25]:
df_va['VA'] = df_va['VA'].apply(nettoyer_texte)

In [26]:
df_va['Nom'] = listeNom
df_va['Description'] = OriginDescription

In [27]:
df_va

,ID,VA,Nom,Description
0,597,file system resources,Absolute Path Traversal,An adversary with access to file system resour...
1,1,Improperly specified ACL,Accessing Functionality Not Properly Constrain...,"In applications, particularly web applications..."
2,31,http cookies store,Accessing/Intercepting/Modifying HTTP Cookies,This attack relies on the use of HTTP Cookies ...
3,575,functionality mean identify,Account Footprinting,An adversary exploits functionality meant to i...
4,173,interface,Action Spoofing,An adversary is able to disguise one action fo...
...,...,...,...,...
554,32,value parameters without,XSS Through HTTP Query Strings,An adversary embeds malicious script code in t...
555,199,alternate form keywords,XSS Using Alternate Syntax,An adversary uses alternate forms of keywords ...
556,245,input validation use,XSS Using Doubled Characters,The adversary bypasses input validation by usi...
557,247,application filter input,XSS Using Invalid Characters,An adversary inserts invalid characters in ide...


In [28]:
df_va.to_csv("Tableau.csv", index=False)